### Paso 1: Dataset de Salud

In [ ]:
import pandas as pd
import tempfile
import os

In [ ]:
df = pd.read_csv("patients.csv")

In [ ]:
df.drop(columns=["id"], inplace=True)

In [ ]:
# 2. Guardar en un archivo temporal sin 'id'
with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as tmp:
    temp_path = tmp.name

In [ ]:
df.to_csv(temp_path, index=False)

### Paso 2: Subir archivo a Google Cloud Storage

In [ ]:
from google.cloud import storage
client = storage.Client()
bucket_name = "salud-lab-demo"
bucket = client.create_bucket(bucket_name, location="US")
blob = bucket.blob("datasets/patients_no_id.csv")
blob.upload_from_filename(temp_path)

In [ ]:
os.remove(temp_path)

### Paso 3: Crear dataset y tabla en BigQuery

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client()
project_id = bq_client.project
dataset_id = "healthcare_analysis"
dataset_ref = bigquery.Dataset(f"{project_id}.{dataset_id}")
bq_client.create_dataset(dataset_ref, exists_ok=True)

schema = [
    bigquery.SchemaField("age", "INT64"),
    bigquery.SchemaField("sex", "STRING"),
    bigquery.SchemaField("dataset", "STRING"),
    bigquery.SchemaField("cp", "STRING"),
    bigquery.SchemaField("trestbps", "FLOAT64"),
    bigquery.SchemaField("chol", "FLOAT64"),
    bigquery.SchemaField("fbs", "STRING"),
    bigquery.SchemaField("restecg", "STRING"),
    bigquery.SchemaField("thalch", "FLOAT64"),
    bigquery.SchemaField("exang", "STRING"),
    bigquery.SchemaField("oldpeak", "FLOAT64"),
    bigquery.SchemaField("slope", "STRING"),
    bigquery.SchemaField("ca", "FLOAT64"),
    bigquery.SchemaField("thal", "STRING"),
    bigquery.SchemaField("num", "INT64")    
]

table_id = f"{project_id}.{dataset_id}.patients"
table = bigquery.Table(table_id, schema=schema)
bq_client.create_table(table, exists_ok=True)

### Paso 4: Cargar datos desde GCS a BigQuery

In [ ]:
uri = f"gs://{bucket_name}/datasets/patients_no_id.csv"
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    schema=schema,
)
load_job = bq_client.load_table_from_uri(uri, table_id, job_config=job_config)
load_job.result()

### Paso 5: Consultas SQL en BigQuery

In [ ]:
query = f"""
SELECT cp, COUNT(*) as pacientes, AVG(chol) as avg_chol
FROM `{table_id}`
GROUP BY cp
ORDER BY avg_chol DESC
"""
df = bq_client.query(query).to_dataframe()
print(df)

### Paso 6: Optimización con particionado y clustering

In [ ]:
from google.cloud import bigquery

clustering_schema = schema  # mismo esquema
clustering_table = bigquery.Table(table_id, schema=clustering_schema)
clustering_table.clustering_fields = ["cp"]
bq_client.update_table(clustering_table, ["clustering_fields"])

### Paso 7: Modelo con Vertex AI

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=project_id, location="us-central1")

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="DatasetPacientes",
    bq_source=f"bq://{table_id}"
)

In [ ]:
training_job = aiplatform.AutoMLTabularTrainingJob(
    display_name="modelo-num-minlogloss",
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

In [ ]:
model = training_job.run(
    dataset=dataset,
    target_column="num",
    model_display_name="modelo_outcome_autoML",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)